[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Gestión de formularios.

Flask permite desplegar, validar y obtener la información que se ingresa mediante las formas de un documento HTML. La gestión de formas corre a cargo de la extensión *flask-wtf*, la cual deriva del paquete *wtforms*.

Para instalar los modulos en cuestión es necesario intalar el paquete *flask-wtf*.

In [ ]:
!pip install flask-wtf

## El paquete *wtforms*.
WTForms es es un proyecto que permite crear y validar formas en HTML. Para mayor información puede visitar el sitio https://wtforms.readthedocs.io/en/latest.

In [ ]:
import wtforms

In [ ]:
help(wtforms)

### Los elementos básicos de *wtfoms* son:
* **Campos**, los cuales capturan un tipo de dato específico y se definen mediante la clase ```Field```. Cada instancia de ```Field``` puede ser definido mediante diversos parámetros como validadores, identificador, descripción, errores, etc.
* **Formularios**, las cuales son una colección de campos que pueden ser accedidos utilizando la sintaxis de un objeto _dict_ o mediante el operador de atributo "*.*" y están definidas por la clase *Form*.
* **Validadores**, los cuales permiten corroborar que la información que se ingresa se ajusta a un requerimiento específico.
* ***Widgets***, las cuales son las herramientas encargadas de generar el elemento HTML de cada campo.

#### Campos.
 La clase *Field* puede definir diversos parámetros al instanciarse y cuenta con el método métodos para validar y procesar la información que recibe. El método *validate()* es el método encargado de hacer las evaluaciones de los datos asignados con respecto a los validadores definidos.

In [ ]:
dir(wtforms.Field)

Existen diversos tipos de clases que corresponde a diversos tipos de campos, los cuales pueden ser consultados de la siguiente manera:

In [ ]:
dir(wtforms.fields)

Un objeto de campo se define instanciando la clase del tipo al que corresponde de la siguente forma:
```
<nombre> = <tipo de campo>(<descripción de tipo str>, [<lista de validadores>], default=<valor predeterminado>)
```
Los campos no pueden ser gestionados a menos que se encuentren dentro de una forma.

#### Validadores.
Los validadores son un conjunto de clases que corroboran ciertas reglas definidas, dependiendo del tipo de información que se requiera.

In [ ]:
help(wtforms.validators)

**Ejemplo:**

Se creará un objeto a partir de la clase *StringField* utilizando el validador *Length* para asegurarse que los campos *nombre* y *primer apellido* no sean cadenas de caracteres vacías.

In [ ]:
from wtforms import StringField
from wtforms.validators import Length

El validador *Lenght* acepta entre otros, al parámetro *min*, el cual designa el tamaño mínimo del dato ingresado como cadena de caracteres. 

In [ ]:
nombre = StringField('nombre', [Length(min=1)], default='Juan')

In [ ]:
nombre

El identificador *nombre* corresponde a un objeto *UnboundField*, lo cual significa que es un campo que no está ligado a una forma.

#### Formas.
La clase *Form* es menos compleja que la clase *Field*, pero tambien cuenta con parámetros de configuración y métodos de proceso y validación de datos.

Cuando un campo  es defindo en una forma, este adquiere el atributo *data*, en el que se almacena la información que captura.

In [ ]:
help(wtforms.Form)

### Ejemplo del uso de *wtforms*.
* Se creará la clase *NombreEstudiante* que es subclase de *Form*, la cual incluirá los campos son de tipo *StringField*, corespondientes a *nombre*, *primer_apellido* y *segundo_apellido*.
* El valor por defecto de todos los campos es una cadena de caracteres vacía.
* Los campos *nombre* y *primer_apellido* incluyen al validador *Length*, el cual verificará que los campos no sean cadenas de caracteres vacías.
* Se creará el objeto *forma* a partir de la clase *NombreEstudiante*.

In [ ]:
from wtforms import Form, StringField, validators
class NombreEstudiante(Form):
    nombre = StringField('Nombre', [validators.Length(min=1)], default='')
    primer_apellido = StringField('Primer apellido', [validators.Length(min=1)], default='')
    segundo_apellido = StringField('Segundo apellido', default='')

In [ ]:
forma = NombreEstudiante()

In [ ]:
forma['nombre']

In [ ]:
forma.nombre

El campo *forma* en este caso corresponde a un objeto *wtforms.fields.core.StringField*, lo que indica que dicho campo está ligado a una forma. En este caso, el campo adquiere el atributo *data*.

In [ ]:
forma.nombre.data

El método *validate()* del objeto *Form* realizará las validaciones de cada campo y en caso de una falla de validación regresará *False* y añadirá la descripción de cada falla de validación al atributo *errors*.

In [ ]:
forma.validate()

In [ ]:
forma.errors

In [ ]:
forma.nombre.data = 'Juan'
forma.primer_apellido.data = 'Pérez'

In [ ]:
forma.validate()

In [ ]:
forma.errors

## El módulo *flask_wtf*.
El módulo *flask_wtf* es la implementación del módulo *wtforms* para *Flask* y contiene a la clase _FlaskForm_).

In [ ]:
from flask_wtf import FlaskForm

In [ ]:
help(FlaskForm)

### Métodos propios de la clase *FlaskForm*.
La clase *FlaskForm* define los métodos:

* *hidden_tag()*
* *is_submitted()*
* *validate_on_submit()*

#### El método *hidden_tag()*.
Este método se utiliza para agregar una etiqueta oculta dentro de una forma. Porm lo general se utiliza para evitar ataques de tipo CSRF.

#### El método *is_submitted()*.
Este método valida si la forma fue enviada.

#### El método *validate_on_submit()*.
Este método valida los datos de cada campo una vez que son enviados y regresa _True_ en caso de que dicha validación haya sido exitosa.

### Protección contra ataques CSRF.

El paquete *wtforms* utiliza por defecto un mecanismo que impide que reciba peticiones que no pertenezcan al sitio. A esta técnica de penetración se le conoce como "CSRF", por las siglas en inglés de Cross Site Request Forgery o Falsificación de Peticiones Cruzadas entre Sitios.

#### Configuración de la clave de cifrado de un objeto *Flask*.

Esta protección se realiza mendiante el uso de claves de cifrado a partir de las cuales generan "tokens" que verifican la autenticidad de las peticiones. La clave de cifrado de un objeto *Flask* se ingresa mediante la asignación de dicha clave al campo *'SECRET_KEY'* del atributo *config* de la siguiente manera.

``` python
<objeto Flask>.config['SECRET_KEY'] = <objeto tipo str>
```

### El atributo *config* de los objetos *Flask*.

Este atributo corresponde a un objeto de tipo _dict_ el cual contiene variables de configuración de la aplicación.

In [ ]:
from flask import Flask
app = Flask(__name__)
print(app.config)

### Ejemplo del uso de *flask_wtf*.

* Se creará la clase *NombreEstudiante* que es subclase de *FlaskForm*, la cual incluirá lo siguiente:
    * Los campos *nombre*, *primer_apellido*, y *segundo_apellido* son instancias de *StringField* y su valor por defecto es una cadena de caracteres vacía.  
    * Los campos *nombre* y *primer_apellido* incluyen al validador *Length*, el cual verificará que los campos no sean cadenas de caracteres vacías.
    * El campo *enviar* es instancia de *SubmitField*.
* Se creará el objeto *app* instanciándolo de *Flask*.
* Se asignará una cadena de caracteres al elemento 'SECRET_KEY'del atributo *app.config*.
* Se creará el objeto *forma* a partir de la clase *NombreEstudiante*.
* Se creará la función de vista *index()* correspondiente a la ruta raíz del servidor, aceptando los métodos POST y GET. Dicha función realizará lo siguiente:
    * Instanciará al objeto *forma* a partir de la clase *NombreEstudiante*.
    * Desplegará una página HTML mendiate la función *render_template()* a partir de la plantilla *plantilla_formas.html*, asignando al parámetro *form* el objeto *forma*.
    * En caso de que la forma haya sido enviada desde un cliente con todas las validaciones correctas, desplegará el contenido capturado por cada elemento del objeto *forma*.
* La plantilla [*templates/plantilla_formas.html*](templates/plantilla_formas.html) contiene lo siguiente:
``` html
<h1> Datos personales del alumno </h1>
<form method="POST">
    {{ form.hidden_tag() }}
    {{ form.nombre.label }} {{form.nombre(id='nombre')}}
    {{ form.primer_apellido.label }} {{form.primer_apellido(id='primer apellido')}}
    {{ form.segundo_apellido.label }} {{form.segundo_apellido(id='segundo apellido')}}
    {{ form.enviar }}
</form>
```

Una vez que ejecute la siguiente celda, podrá acceder a la aplicación localizada en http://localhost:5000/

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import Length
from flask import Flask, render_template


class NombreEstudiante(FlaskForm):
    nombre = StringField('Nombre', [Length(min=1)], default='')
    primer_apellido = StringField('Primer apellido', [Length(min=1)], default='')
    segundo_apellido = StringField('Segundo apellido', default='')
    enviar = SubmitField('Enviar')
    
app = Flask(__name__)
app.config['SECRET_KEY']='Saludines'


@app.route('/', methods=['GET', 'POST'])
def index():
    forma = NombreEstudiante()
    if forma.validate_on_submit():
        for dato in forma:
            print(dato) 
    return render_template('plantilla_formas.html', form=forma)

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

## Patrón de validación de datos mediante un macro de Jinja.

A lo largo del tiempo, los desarrolladores de aplicaciones basadas en Flask han identificado ciertos patrones que pueden ser útiles. Uno de ellos corresponde a un macro en Jinja que despliega los mensajes de error de validación de cada campo. Puede saber más al respecto en la siguiente liga:

http://flask.pocoo.org/docs/0.12/patterns/wtforms/#forms-in-templates

El código de este patrón es el siguiente y se ha guardado en el archivo [*templates/_formhelpers.html*](templates/_formhelpers.html).

``` html
{% macro render_field(field) %}
  <dt>{{ field.label }}
  <dd>{{ field(**kwargs)|safe }}
  {% if field.errors %}
    <ul class=errors>
    {% for error in field.errors %}
      <li>{{ error }}</li>
    {% endfor %}
    </ul>
  {% endif %}
  </dd>
{% endmacro %}
```
El macro *render_field()* desplegará los errores de validación de cada campo al presionar el botón de envío.



**Ejemplo:**

Se creará el archivo [*templates/pantilla_formas_avanzadas.html*](templates/plantilla_formas_avanzadas.html) que incluye el siguiente código, que es similar a la plantilla *templates/pantilla_formas.html*, pero importando y aplicando el macro *render_field()*.

``` html
<h1> Datos personales del alumno </h1>
{% from "_formhelpers.html" import render_field %}
<form method="POST">
    {{ form.hidden_tag() }}
    {{ render_field(form.nombre) }}
    {{ render_field(form.primer_apellido) }}
    {{ render_field(form.segundo_apellido) }}
    {{ form.enviar }}
</form>
```
**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import Length
from flask import Flask, render_template


class NombreEstudiante(FlaskForm):
    nombre = StringField('Nombre', [Length(min=1)], default='')
    primer_apellido = StringField('Primer apellido', [Length(min=1)], default='')
    segundo_apellido = StringField('Segundo apellido', default='')
    enviar = SubmitField('Enviar')
    
app = Flask(__name__)
app.config['SECRET_KEY']='Saludines'


@app.route('/', methods=['GET', 'POST'])
def index():
    forma = NombreEstudiante()
    if forma.validate_on_submit():
        for dato in forma:
            print(dato) 
    return render_template('plantilla_formas_avanzadas.html', form=forma)

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

## Creación de validadores propios.

Cuando un dato no cumple con las condiciones de un validador, dicho validador levanta una excepción de tipo *ValidatorError*, la cual es "cachada" por el objeto *Field* y el mensaje de error es guardado en el atributo *errors*.

Es posible crear validadores propios definiendo funciones que levanten un error de tipo *ValidatorError*.

**Ejemplo:**

La siguiente aplicación pedirá los datos personales de un alumno por medio de una página web, utilizando diversos tipos de campos. Sin embargo, el campo promedio requiere de una validación adicional que garantice que se ingresó un número real entre 0 y 10. 

La función *valida_promedio()* validará que se ingrese un número del 0 al 10 y se añadirá a la lista de validadores del campo *promedio*. 

Una vez que ejecute la siguiente celda, podrá acceder a la aplicación localizada en http://localhost:5000/

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, BooleanField, SelectField
from wtforms.validators import DataRequired, ValidationError
from flask import Flask, render_template

carreras = (('Derecho', 'Derecho'), ('Medicina', 'Medicina'), ('Sistemas', 'Sistemas'), ('Diseño', 'Diseño'))

        
class DatosEstudiante(FlaskForm):

            
    def valida_promedio(form, field):
        try:
            numero = float(field.data)
        except:
            raise ValidationError('Debe de ingresar un número')
        if numero < 0 or numero > 10:
            raise ValidationError('Debe de ingresar un número entre 0 y 10')        
            
            
    nombre = StringField('Nombre', [DataRequired()], default = '')
    primer_apellido = StringField('Primer apellido', [DataRequired()], default = '')
    segundo_apellido = StringField('Segundo apellido', default = '')
    carrera = SelectField('Carrera', [DataRequired()], choices = carreras)
    semestre = SelectField('Semestre', [DataRequired()], choices = [(str(x), str(x)) for x in range(1, 50)])
    promedio = StringField('Promedio', [DataRequired(), valida_promedio], default = '0')
    alcorriente = BooleanField('Al corriente de pagos')
    enviar = SubmitField('Enviar')
    
            
app = Flask(__name__)
app.config['SECRET_KEY']='Saludines'


@app.route('/', methods=['GET', 'POST'])
def index():
    forma = DatosEstudiante()
    if forma.validate_on_submit():
        for campo in ['nombre', 'primer_apellido', 'segundo_apellido', 'carrera', 'semestre', 'promedio', 'alcorriente']:
            print(forma[campo].data) 
    return render_template('alumno_carrera.html', form=forma)

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>